In [25]:
from datetime import datetime
from elasticsearch import Elasticsearch
import os
import sys
import numpy as np

inputs = sys.argv
#inputs = [0, 'tag:"kubernetes.var.log.containers.lgms*" AND ProductOutboundCreationJob',"06/09/2017 16:30", "06/09/2017 17:30", 'log.txt']
query = inputs[1]
startQuery = inputs[2]
endQuery = inputs[3]
output_path = inputs[4]

In [13]:
es = Elasticsearch(['http://es.ops.lzd.co'], port=80)

In [22]:
end = datetime.strptime(endQuery, "%d/%m/%Y %H:%M")##("06/09/2017 16:30", "%d/%m/%Y %H:%M")
start = datetime.strptime(startQuery, "%d/%m/%Y %H:%M")
queryFrom = 0
size = 10
timerange = {"range": {"@timestamp": {"gte": int(start.timestamp()*1000),"lte": int(end.timestamp()*1000),"format": "epoch_millis"}}}
params = {"from" : queryFrom, "size" : size, "query": {"bool": {"must":[{"query_string": {"analyze_wildcard": True,"query": query}}, timerange]}}}

In [23]:
##query = 'tag:"kubernetes.var.log.containers.lgms*"'


In [29]:
writer = open(output_path, 'w')
pages = np.ceil(res['hits']['total']/size)
for i in range(0, int(pages)):
    params['from'] = i * size
    res = es.search(index='lgms-app*', body=params)
    for log in res['hits']['hits']:
        writer.write(str(log))

In [24]:
print(res)

{'took': 75, 'timed_out': False, '_shards': {'total': 75, 'successful': 75, 'failed': 0}, 'hits': {'total': 146, 'max_score': 11.497336, 'hits': [{'_index': 'lgms-app-2017.09.06', '_type': 'fluentd', '_id': 'AV5Wh4PsVJuXGbOMdB79', '_score': 11.497336, '_source': {'log': '[2017-09-06 09:31:11] Processed: Lgms\\Jobs\\ProductOutbound\\ProductOutboundCreationJob\n', 'stream': 'stdout', 'tag': 'kubernetes.var.log.containers.lgms-product-outbound-creation-2685167734-vmhcg_lgms_product-outbound-creation-333c914cb00ece7d5f4fcca800583011d742dd908be9682ec5abfaa95660ab7d.log', '@timestamp': '2017-09-06T09:31:11+00:00'}}, {'_index': 'lgms-app-2017.09.06', '_type': 'fluentd', '_id': 'AV5WoJ6ZhXlO8HT5-GS-', '_score': 11.497336, '_source': {'log': '[2017-09-06 09:58:38] Processed: Lgms\\Jobs\\ProductOutbound\\ProductOutboundCreationJob\n', 'stream': 'stdout', 'tag': 'kubernetes.var.log.containers.lgms-product-outbound-creation-2685167734-vmhcg_lgms_product-outbound-creation-333c914cb00ece7d5f4fcca800